# DATA TRANSFORMATION - 16_medicamentos
## 0. Previos

Se cargan las bases y las librerías a utilizar.

In [1]:
# Librerías
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import numpy as np

import generic_funcions as gf

ruta_archivos = 'C:/Users/monic/documentos/ds4a/project/datos/'

## 1. Load

In [2]:
modulo = 'Medicamentos'

In [3]:
dcc = gf.diccionario_llaves()

In [ ]:
base = gf.carga_datos( ruta = ruta_archivos, diccionario = dcc, modulo = modulo )
base.head()

## 2. Transform

In [ ]:
base = gf.letra_codigo( base, 'Codigo_Diagnostico_EPS_Op' )
base.head()

* Number of delivered doses per month
* Number of different diseases per month

In [ ]:
base_ = base.groupby(['id','year','month','Codigo_Diagnostico_EPS_Op_cod'])['Numero_Cantidad_Prestaciones'].sum().\
    reset_index( name = 'num_doses' )
base_['num_dis'] = 1
base_.head()

In [ ]:
# all diagnoses different form j will be in the same category
base_['diag'] = base_['Codigo_Diagnostico_EPS_Op_cod'].apply( lambda x : 'j' if x == 'j' else 'otra' )

In [ ]:
# pivot table to have separate variables, nans replaced with zero
base_p = base_.pivot_table(index=['id','year','month'], columns='diag', values=['num_doses','num_dis'], aggfunc=np.sum).reset_index()
base_p.columns = ['_'.join(col).strip() for col in base_p.columns.values]
base_p.fillna( 0 , inplace = True)
base_p.head()

* Flag variables

In [ ]:
base_p['num_dis'] = base_p['num_dis_otra'] + base_p['num_dis_j']
base_p['flag_otra'] = base_p['num_dis_otra'].apply( lambda x : 0 if x == 0 else 1 )
base_p.rename( columns={ 'num_dis_j':'flag_j' }, inplace=True)
base_p.head()

In [ ]:
# add prefix
base_p.columns = ['med_' + s for s in base_p.columns]

In [ ]:
base_p.rename( columns={ 'med_id_':'id', 'med_year_':'year', 'med_month_':'month' }, inplace=True)
base_p.head()

## 3. Merge with ids table

In [ ]:
ids_mensual = gf.base_ids_mensual( ruta_archivos )
base_final_med = ids_mensual.merge( base_p, how='left')
base_final_med.head()